In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lecture 3B - Data Integration*

# Table of Contents
* [Lecture 3B - Data Integration*](#Lecture-12---Data-Integration*)
	* &nbsp;
	* [Content](#Content)
	* [Learning Outcomes](#Learning-Outcomes)
	* [Integration of data from multiple sources](#Integration-of-data-from-multiple-sources)
	* [Merging Datasets](#Merging-Datasets)
		* [Database-style DataFrame Merges](#Database-style-DataFrame-Merges)
		* [Merging on Index](#Merging-on-Index)
		* [Concatenating Data Frames](#Concatenating-Data-Frames)
		* [Concatenation on axes](#Concatenation-on-axes)
		* [Updating Columns](#Updating-Columns)
			* [Combining together values within Series or DataFrame columns from different sources](#Combining-together-values-within-Series-or-DataFrame-columns-from-different-sources)


---

### Content

1. dataset merging
2. dataset concatenation
3. dataset value updating


\* Content in this notebook is based on the material in the "Python for Data Analysis" book by Wes McKinney, chapter 7. and material from http://pandas.pydata.org/

### Learning Outcomes

At the end of this lecture, you should be able to:

* describe the inner, outer, left, right join-types for merging dataframes 
* merge different dataframes on indices or common columns
* concatenate dataframes horizontally or vertically
* update values in one dataframe based on values from a similar dataframe 

---

In [2]:
from IPython.display import HTML, IFrame
IFrame("http://pandas.pydata.org/pandas-docs/dev/merging.html", width=1100, height=500)

## Integration of data from multiple sources

Much of the work in the overall analytics pipeline is spent on data preparation: loading, cleaning, transforming, and rearranging. The total time spent on this task can be up to 90% of the entire analytics project time resources, before any actual useful 'analytics' work is done.

Increasingly datasets from multiple sources must be integrated into a single dataset. This can be a difficult task especially if done manually through Excel-type programs. In many cases it is impossible due to file size, and often undesirable to to the fact that it is difficult to document the process and also impossible to audit and repeat automatically.

Many analytics professionals choose to do ad hoc processing of data from one form to another using a general purpose programming, like Python, Perl, R, or Java, or UNIX text processing tools like sed or awk. 

Fortunately, pandas along with the Python standard library provide you with a high-level, flexible, and high-performance set of core manipulations and algorithms to enable you to integrate and wrangle data into a single source without much trouble.

## Merging Datasets

Data contained in pandas objects can be combined together in a number of built-in ways:

* `pandas.merge` connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
* `pandas.concat` glues or stacks together objects along an axis (`axis=1` columns, `axis=0` rows).

### Database-style DataFrame Merges

Merge or join operations combine data sets by linking rows using one or more **keys**.
These operations are central to relational databases. 

The `merge` function in pandas is
the main entry point for using these algorithms on your data.


We will begin with simple examples:

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pylab import rcParams

# Set some Pandas options as you like
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 30)

rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20

%matplotlib inline

In [4]:
df1 = pd.DataFrame(
                    {'name': ['ben', 'ben', 'adam', 'cindy', 'adam', 'adam', 'ben'],
                     'transaction': np.random.randint(1, 50, 7)}
                  )

df2 = pd.DataFrame(
                    {'name': ['adam', 'ben', 'darren'],
                     'age': [33,25,40]}
                )
print(df1)
print('---------------')
print(df2)

    name  transaction
0    ben           39
1    ben           36
2   adam           31
3  cindy           13
4   adam           20
5   adam           31
6    ben           15
---------------
     name  age
0    adam   33
1     ben   25
2  darren   40


Below is an example of a many-to-one merge situation using the `pandas.merge` method; the data in `df1` has multiple rows
labelled 'adam' and 'ben', whereas `df2` has only one row for each value in the key column. Calling
merge with these objects we obtain:

In [5]:
pd.merge(df1, df2)

,name,transaction,age
0,ben,39,25
1,ben,36,25
2,ben,15,25
3,adam,31,33
4,adam,20,33
5,adam,31,33


Note that we **did not specify** which column to join on. If not specified, merge uses the
**overlapping column names as the keys**. It is however good practice to specify explicitly:

In [6]:
pd.merge(df1, df2, on='name')

,name,transaction,age
0,ben,39,25
1,ben,36,25
2,ben,15,25
3,adam,31,33
4,adam,20,33
5,adam,31,33


Notice that original indexes cannot be preserved when merging on columns.

If the column names are different in each object, you can specify them separately:

In [7]:
df3 = pd.DataFrame(
                {'lkey': ['ben', 'ben', 'adam', 'cindy', 'adam', 'adam', 'ben'],
                 'data1': np.random.randint(1, 50, 7)}
                )

df4 = pd.DataFrame(
                {'rkey': ['adam', 'ben', 'darren'],
                 'age': [33,25,40]}
                )

print(df3)
print('---------')
print(df4)

    lkey  data1
0    ben     41
1    ben     21
2   adam      5
3  cindy     21
4   adam      4
5   adam     38
6    ben     28
---------
     rkey  age
0    adam   33
1     ben   25
2  darren   40


In [8]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,age
0,ben,41,ben,25
1,ben,21,ben,25
2,ben,28,ben,25
3,adam,5,adam,33
4,adam,4,adam,33
5,adam,38,adam,33


You probably noticed that the 'cindy' and 'darren' values and associated data are missing from
the result. 

**By default merge does an 'inner' join**; the keys in the result are the **intersection** of the two sets.
Other possible options are 'left', 'right', and 'outer'. 

The outer join takes the
union of the keys, combining the effect of applying both left and right joins:

In [9]:
print(df1)
print('---------')
print(df2)

    name  transaction
0    ben           39
1    ben           36
2   adam           31
3  cindy           13
4   adam           20
5   adam           31
6    ben           15
---------
     name  age
0    adam   33
1     ben   25
2  darren   40


In [10]:
pd.merge(df1, df2, how='outer')

,name,transaction,age
0,ben,39.0,25.0
1,ben,36.0,25.0
2,ben,15.0,25.0
3,adam,31.0,33.0
4,adam,20.0,33.0
5,adam,31.0,33.0
6,cindy,13.0,NaN
7,darren,NaN,40.0


The above merges have been examples of **one-to-many and many-to-one merges**. Sometimes it is necessary to perform **one-to-one merges on indexes**, these we perform on indexes and we will see them later.

Many-to-many merges have well-defined though not necessarily intuitive behaviour.

Here’s an example:

In [11]:
df1 = pd.DataFrame(
                {'name': ['ben', 'ben', 'adam', 'cindy', 'adam', 'ben'],
                 'transaction_1': range(6)}
                )
df2 = pd.DataFrame(
                {'name': ['adam', 'ben', 'adam', 'ben', 'darren'],
                 'transaction_2': range(5)}
                )
print(df1)

print('---------')
print(df2)

    name  transaction_1
0    ben              0
1    ben              1
2   adam              2
3  cindy              3
4   adam              4
5    ben              5
---------
     name  transaction_2
0    adam              0
1     ben              1
2    adam              2
3     ben              3
4  darren              4


In [12]:
pd.merge(df1, df2, on='name', how='left')

,name,transaction_1,transaction_2
0,ben,0,1.0
1,ben,0,3.0
2,ben,1,1.0
3,ben,1,3.0
4,adam,2,0.0
5,adam,2,2.0
6,cindy,3,NaN
7,adam,4,0.0
8,adam,4,2.0
9,ben,5,1.0


Many-to-many joins form the **Cartesian product** of the rows. Since there were 3 'ben'
rows in the left DataFrame and 2 in the right one, there are 6 'ben' rows in the result.

The join method only affects the distinct key values appearing in the result. 

By this we mean that if there are unique keys in either left or right hand side, the type of join method will determine if rows with the unique values appear in the final result:

In [14]:
pd.merge(df1, df2, how='inner')

,name,transaction_1,transaction_2
0,ben,0,1
1,ben,0,3
2,ben,1,1
3,ben,1,3
4,ben,5,1
5,ben,5,3
6,adam,2,0
7,adam,2,2
8,adam,4,0
9,adam,4,2


**Exercises**: For the following exercises, use the 3 data sets below (source http://www.goldpriceoz.com/). The datasets below represent the "Gold Price Annual End of Period" for a selection of currencies.

Create dataframes from the datasets below by highlighting the dataset and right-clicking copy, followed by the execution of the following line: 

df = pd.read_clipboard() 

**Exercise**: Your first task is to merge the Year End Period data with each of the 2 datasets containing the period end price of gold. Call them df_USD and df_AUD.

In [13]:
# df_y = pd.read_clipboard()
df_y

,DATAFRAME,1
0,ID,Year
1,0,1999
2,1,2000
3,2,2001
4,3,2002
5,4,2003
6,5,2004
7,6,2005
8,7,2006
9,8,2007


In [14]:
# df_USD = pd.read_clipboard()
df_USD

,DATAFRAME,2,(df_USD)
ID,USD,EUR,GBP
0,290.2,289.6,180.1
1,274.5,292.3,183.7
2,276.5,310.5,189.9
3,347.2,330.9,215.7
4,416.3,330.0,232.5
5,435.6,320.5,226.9
6,513.0,434.9,298.8
7,632.0,479.3,322.9
8,833.8,570.3,418.8


In [15]:
# df_AUD = pd.read_clipboard()
df_AUD

,DATAFRAME,3,(df_AUD)
ID,AUD,CNY,INR
0,443.5,2403.1,12630.9
1,493.9,2271.7,12811.3
2,540.1,2288.5,13332.8
3,616.6,2873.8,16648.2
4,552.5,3445.2,18991.4


**Exercise**: Merge df_USD and df_AUD so that only common data to both datasets is preserved in the result.

In [22]:
pd.merge(df_USD,df_AUD,how="inner")

,DATAFRAME,2,(df_USD),3,(df_AUD)


**Exercise**: Merge df_USD and df_AUD so that all data in df_USD is preserved in the result.

In [20]:
pd.merge(df_USD,df_AUD,how='left')

,DATAFRAME,2,(df_USD),3,(df_AUD)
0,USD,EUR,GBP,NaN,NaN
1,290.2,289.6,180.1,NaN,NaN
2,274.5,292.3,183.7,NaN,NaN
3,276.5,310.5,189.9,NaN,NaN
4,347.2,330.9,215.7,NaN,NaN
5,416.3,330.0,232.5,NaN,NaN
6,435.6,320.5,226.9,NaN,NaN
7,513.0,434.9,298.8,NaN,NaN
8,632.0,479.3,322.9,NaN,NaN
9,833.8,570.3,418.8,NaN,NaN


**Exercise**: Merge df_USD and df_AUD so that all data in df_AUD is preserved in the result.

In [21]:
pd.merge(df_USD,df_AUD,how='right')

,DATAFRAME,2,(df_USD),3,(df_AUD)
0,AUD,NaN,NaN,CNY,INR
1,443.5,NaN,NaN,2403.1,12630.9
2,493.9,NaN,NaN,2271.7,12811.3
3,540.1,NaN,NaN,2288.5,13332.8
4,616.6,NaN,NaN,2873.8,16648.2
5,552.5,NaN,NaN,3445.2,18991.4


**Exercise**: Merge df_USD and df_AUD so that all data from both datasets is preserved in the result.

In [24]:
pd.merge(df_USD,df_AUD,how="outer")

,DATAFRAME,2,(df_USD),3,(df_AUD)
0,USD,EUR,GBP,NaN,NaN
1,290.2,289.6,180.1,NaN,NaN
2,274.5,292.3,183.7,NaN,NaN
3,276.5,310.5,189.9,NaN,NaN
4,347.2,330.9,215.7,NaN,NaN
5,416.3,330.0,232.5,NaN,NaN
6,435.6,320.5,226.9,NaN,NaN
7,513.0,434.9,298.8,NaN,NaN
8,632.0,479.3,322.9,NaN,NaN
9,833.8,570.3,418.8,NaN,NaN


**Exercise**: Plot the price of gold for one of the currencies, for each of the years in the dataset using an appropriate figure type.

We can merge with multiple keys. To merge with multiple keys, pass a list of column names:

In [25]:
left = pd.DataFrame(
                 {'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'one'],
                 'lval': [1, 2, 3]}
                 )

right = pd.DataFrame(
                  {'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]}
                  )
print(left)
print('---------')
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
---------
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [26]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combinations will appear in the result depending on the choice
of merge method, **think of the multiple keys as forming an array of tuples to be used
as a single join key**.

When joining columns-on-columns, the **indexes on the passed Data Frame objects are discarded**.

A last issue to consider in merge operations is the treatment of overlapping column
names. While you can address the overlap manually, merge has a suffixes option for specifying strings to append to overlapping
names in the left and right DataFrame objects:

In [19]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Notice the suffixes '_x' and '_y' above which are default. We can explicitly specify them:

In [28]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


**Exercise**: Given the following:

In [29]:
df5 = pd.DataFrame(
                 {'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'one'],
                 'val': [1, 2, 3]}
                 )

df6 = pd.DataFrame(
                  {'key1': ['one', 'one', 'one', 'two'],
                   'key2': ['foo', 'foo', 'bar', 'bar'],
                   'val': [4, 5, 6, 7]}
                  )
print(df5)
print('---------')
print(df6)

  key1 key2  val
0  foo  one    1
1  foo  two    2
2  bar  one    3
---------
  key1 key2  val
0  one  foo    4
1  one  foo    5
2  one  bar    6
3  two  bar    7


Your task is to merge on key1 from df5 and key2 from df6 using a merge type that preserves all unique keys, and renaming overlapping columns with the '_l' and '_r' suffixes.

In [30]:
pd.merge(left, right, on=['key1', 'key2'], suffixes=('_l', '_r'))

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


---

### Merging on Index

In some instances, the merge key or keys in a DataFrame will be found in its index. In this
case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the
index should be used as the merge key:

In [31]:
left1 = pd.DataFrame(  {'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                        'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, 
                        index=['a', 'b'])

print(left1)
print('---------')
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
---------
   group_val
a        3.5
b        7.0


In [32]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Once again, since the default merge method is to intersect the join keys, you can instead form the
union of them with an outer join:

In [26]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


DataFrame has a more **convenient join method for merging by index**. It can also be
used to combine together many DataFrame objects **having the same or similar indexes
but non-overlapping columns**. In this example, by merging on unique indexes, we will be performing **one-to-one merge** operations.


In [33]:
right2 = pd.DataFrame(
                     { 'group_val' : [10,20]
                      }, index=[1,2]
                     )
print(left1)
print('---------')
print(right2)


  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
---------
   group_val
1         10
2         20


In [34]:
left1.join(right2, how='outer')

,key,value,group_val
0,a,0,NaN
1,b,1,10.0
2,a,2,20.0
3,a,3,NaN
4,b,4,NaN
5,c,5,NaN


# TODO

## Merge Exercises:

**Exercise**: Read in the child_mortality_rates and adult_mortality_rates datasets and merge them on appropriate variables, using a meaningful merge technique. Perform data cleaning where necessary.

In [37]:
cm = pd.read_csv('../datasets/child_mortality_rates.csv')
cm.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Neonatal mortality rate (deaths per 1000 live births),Neonatal mortality rate (deaths per 1000 live births).1,Infant mortality rate (per 1000 live births),Infant mortality rate (per 1000 live births).1,Under-five mortality rate (per 1000 live births),Under-five mortality rate (per 1000 live births).1
0,Country,Year,Data Source,Female,Male,Female,Male,Female,Male
1,Albania,2008,DHS,8.4 [4.6-15.1],11.8 [7.0-20.1],15.5 [10.3-23.3],22.4 [15.9-31.6],16.1 [10.8-23.9],27.3 [20.2-36.9]
2,Armenia,2010,DHS,9.6 [5.1-18.0],9.3 [5.0-17.3],20.3 [12.4-33.1],17.0 [11.0-26.3],21.7 [13.6-34.5],21.3 [14.3-31.4]
3,Armenia,2005,DHS,15.5 [9.5-25.4],21.1 [13.5-33.0],25.5 [16.8-38.5],29.0 [20.2-41.4],28.5 [19.3-41.9],36.4 [26.4-49.8]
4,Armenia,2000,DHS,22.8 [15.9-32.8],28.5 [21.7-37.3],42.1 [32.7-54.1],45.3 [36.3-56.4],44.9 [35.3-56.9],50.1 [40.7-61.6]


In [74]:
labels = list(cm.columns.values)
labels

['Unnamed: 0',
 'Unnamed: 1',
 'Unnamed: 2',
 'Neonatal mortality rate (deaths per 1000 live births)',
 'Neonatal mortality rate (deaths per 1000 live births).1',
 'Infant mortality rate (per 1000 live births)',
 'Infant mortality rate (per 1000 live births).1',
 'Under-five mortality rate (per 1000 live births)',
 'Under-five mortality rate (per 1000 live births).1']

**Exercise**: Generate several plots on the above data. Is there anything interesting?

**Exercise**: Read in the adult_mortality_rate_by_cause dataset and merge it with the above dataset on appropriate variables, using a meaningful merge technique. 

In [62]:
amc = pd.read_csv('../datasets/adult_mortality_rate_by_cause.csv')
amc

,Unnamed: 0,Unnamed: 1,Age-standardized mortality rate by cause (per 100 000 population),Age-standardized mortality rate by cause (per 100 000 population).1,Age-standardized mortality rate by cause (per 100 000 population).2,Age-standardized mortality rate by cause (per 100 000 population).3
0,Country,Year,All Causes,Communicable &amp; other Group I,Noncommunicable diseases,Injuries
1,Afghanistan,2012,1378.2,362.7,846.3,169.2
2,Afghanistan,2000,1660.5,574.8,905.5,180.2
3,Albania,2012,766.1,46.5,671.6,48
4,Albania,2000,986.5,75.4,840,71.1
...,...,...,...,...,...,...
340,Yemen,2000,1354.9,640.8,630.5,83.6
341,Zambia,2012,1508.1,764.3,587.4,156.4
342,Zambia,2000,2735.7,1808.4,712,215.3
343,Zimbabwe,2012,1392.7,711.3,598.9,82.5


In [77]:
# amc.iloc[:,1:].plot()


**Exercise**: Finally, read in the total_health_expenditure_by_country_per_year dataset. Attempt to merge it with the above dataset. What are the challenges? How might you work around them? 

In [29]:
th = pd.read_csv('../datasets/total_health_expenditure_peercent_per_capita_of_gdp_by_country_per_year.csv')


---

### Concatenating Data Frames

Concatenation appends data frames and series objects using the `pandas.concat` method. Data frames can be appended either using the axis=0 option (default) whereby rows are added or using the axis=1, whereby columns are added.

In [78]:
np.random.randn(3, 4)

array([[ 1.56223937, -0.47704184, -0.08126515,  0.88609396],
       [-0.53258807,  0.37690626, -0.13305446, -0.05234848],
       [-0.15874106,  0.70875745, -1.25614981,  0.00635082]])

In [79]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

print(df1)
print('----------')
print(df2)

          a         b         c         d
0  0.226485  0.516497 -0.248146  0.790016
1 -0.625073 -0.407196 -1.162581  0.313852
2 -1.271539 -0.142108 -0.130695 -0.646352
----------
          b         d         a
0 -1.095076  1.547120  0.629612
1  0.302400  1.270563  0.876740


In [80]:
pd.concat([df1, df2], sort=False)

,a,b,c,d
0,0.226485,0.516497,-0.248146,0.790016
1,-0.625073,-0.407196,-1.162581,0.313852
2,-1.271539,-0.142108,-0.130695,-0.646352
0,0.629612,-1.095076,NaN,1.547120
1,0.876740,0.302400,NaN,1.270563


The concat method appends data frames and is not concerned with creating multiple indexes. If the indexes are relevant to the data frame and it is desirable to have unique indexes, then this can be achieved as follows:

In [81]:
pd.concat([df1, df2], ignore_index=True, sort=False)

,a,b,c,d
0,0.226485,0.516497,-0.248146,0.790016
1,-0.625073,-0.407196,-1.162581,0.313852
2,-1.271539,-0.142108,-0.130695,-0.646352
3,0.629612,-1.095076,NaN,1.547120
4,0.876740,0.302400,NaN,1.270563


**Exercise**: Create a Dataframe called df5 having 4 random float values, having a column called 'a', so that it can be appended with column 'a' from df1. Write code to concat df5 with df1.

In [87]:
df5 = pd.DataFrame(np.random.randn(4), columns=['a',])
pd.concat([df1,df5],ignore_index=True,sort=False)

,a,b,c,d
0,0.226485,0.516497,-0.248146,0.790016
1,-0.625073,-0.407196,-1.162581,0.313852
2,-1.271539,-0.142108,-0.130695,-0.646352
3,-0.798464,NaN,NaN,NaN
4,0.630229,NaN,NaN,NaN
5,-0.951942,NaN,NaN,NaN
6,-0.953027,NaN,NaN,NaN


### Concatenation on axes

concat can be used to append on the **column axis**:

In [88]:
df3 = pd.DataFrame(np.random.randn(2, 3), columns=['e', 'f', 'g'])
df3

,e,f,g
0,0.298239,-0.850250,2.589044
1,0.537750,-0.305471,1.181562


In [89]:
df1

,a,b,c,d
0,0.226485,0.516497,-0.248146,0.790016
1,-0.625073,-0.407196,-1.162581,0.313852
2,-1.271539,-0.142108,-0.130695,-0.646352


In [90]:
pd.concat([df1, df3], axis=1)

,a,b,c,d,e,f,g
0,0.226485,0.516497,-0.248146,0.790016,0.298239,-0.850250,2.589044
1,-0.625073,-0.407196,-1.162581,0.313852,0.537750,-0.305471,1.181562
2,-1.271539,-0.142108,-0.130695,-0.646352,NaN,NaN,NaN


The `concat` method is as powerful as the merge, having a number of arguments that allow you produce custom made concatenation types. We can specify the join axes which selects the specified rows:

In [91]:
pd.concat([df1, df3], axis=1 , join_axes=[df1.index[1:3]])

/Users/Tommy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,a,b,c,d,e,f,g
1,-0.625073,-0.407196,-1.162581,0.313852,0.53775,-0.305471,1.181562
2,-1.271539,-0.142108,-0.130695,-0.646352,NaN,NaN,NaN


### Updating Columns

#### Combining together values within Series or DataFrame columns from different sources

Another fairly common situation is to have two like-indexed (or similarly indexed) Series or DataFrame objects and needing to “patch” values in one dataframe with values from another dataframe based on matching indices. 

Here is an example:

In [92]:
df1 = pd.DataFrame([[np.nan, 3., 5.], [-4.6, np.nan, 1], [np.nan, 7., np.nan]])
df2 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5., 1.6, 4]], index=[1, 2])

df1

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,1.0
2,NaN,7.0,NaN


In [93]:
df2

,0,1,2
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0


Say we wanted to update the values in df1, column 2 with those of df1, column 2.

Our intuition might be to do the following:

In [94]:
df1[2] = df2[2]
df1


,0,1,2
0,NaN,3.0,NaN
1,-4.6,NaN,-8.2
2,NaN,7.0,4.0


From the result above you will notice that all the values from df2[2] have been copied over to df1[2], and that all the existing values in df1[2] have been overwritten. In cases where the index row in df1[2] was not found in df2[2], the new value was assigned as NaN.

However, this is not what we wanted. We wanted to copy the values from df2[2], but preserve the values in df1[2] that did not exist in df2[2].

Let's try again

In [95]:
df1 = pd.DataFrame([[np.nan, 3., 5.], [-4.6, np.nan, 1], [np.nan, 7., np.nan]])
df1

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,1.0
2,NaN,7.0,NaN


The function that we need is called update.

In [96]:
df1[2].update(df2[2])
df1

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,-8.2
2,NaN,7.0,4.0


Note that update performs its operation inplace.

What if we now only wanted to update NaN values in df1 with the values in df2 and not just perform a blanket update?

This can be achieved using the combine_first method.  

In [97]:
df1

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,-8.2
2,NaN,7.0,4.0


In [98]:
df2

,0,1,2
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0


In [99]:
df1[[0,1]].combine_first(df2[[0,1]]) 

,0,1
0,NaN,3.0
1,-4.6,NaN
2,-5.0,7.0


Note that this method only takes values from the right DataFrame if they are missing in the left DataFrame.

**Exercise:** Use the datasets below and the command df = pd.read_clipboard() in order to construct dataframes for the exercises below:

**Exercise:** df_USD1 has missing values for the USD and GBP. Populate the missing values with those from the dataframe df_USD2

**Exercise:** df_USD1 has a combination of missing values and erroneous values for the EUR column. Replace all the values in this columns with those that exist in dataframe df_USD2 for this column.